# Lecture 5-3

## Polars

## Week 5 Friday

## Miles Chen, PhD

[https://pola.rs/](https://pola.rs/)

# Why Polars?

Polars is built for speed.

Polars can use lazy execution. The optimizer looks at the entire query and builds an optimized plan for processing it.

## Stick to Pandas for:

- Small/interactive analysis: For quick data exploration and when high performance doesn't matter, pandas is perfectly fine.
- Plotting & convenience: Many plotting libs integrate directly with pandas; Polars usually hands off to pandas or converts to arrays for plotting


## 0) Setup

> If Polars isn't installed yet, run this in the terminal:
```bash
pip install polars
```


In [1]:
import polars as pl
pl.__version__

'1.5.0'


## 1) Load the Titanic dataset

* survived: 0 = No, 1 = Yes
* pclass: Ticket class: 1 = 1st, 2 = 2nd, 3 = 3rd
* sibsp: number of siblings and/or spouses aboard
* parch: number of parents and/or children aboard
* ticket: ticket number
* cabin: cabin number
* embarked: port of embarkation: C = Cherbourg, Q = Queenstown, S = Southampton


In [2]:
# Read in the data eagerly (materializes immediately)
titanic = pl.read_csv("titanic.csv")

titanic

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""
…,…,…,…,…,…,…,…,…,…,…,…
887,0,2,"""Montvila, Rev. Juozas""","""male""",27.0,0,0,"""211536""",13.0,null,"""S"""
888,1,1,"""Graham, Miss. Margaret Edith""","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S"""
889,0,3,"""Johnston, Miss. Catherine Hele…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S"""


## 2) Eager vs. Lazy

- **Eager** executes now and returns results.
- **Lazy** builds a plan (`LazyFrame`) and executes at `.collect()`, enabling optimizations.

The recommended method is to use lazy API. Polars will look at the entire query and optimize it before running. The lazy API allows you to work with datasets that are larger than your computer's memory.

To use the lazy API, use `pl.scan_csv()` instead of `pl.read_csv()`

We'll come back to this topic later, *after* we have introduced operational features in Polars.



## 3) Select and Rename

Select columns (also called series) using `name_of_dataframe.select("column_name")`

Multiple columns can be selected by passing a list of column names.

Columns or series can be renamed using `pl.col("current_name").alias("new_name")`

If you ever want to use attributes or methods for a column, you must use `pl.col("column_name")` and not just the string name.



In [3]:
titanic.select([
    "survived",
    pl.col("pclass").alias("passenger_class"),
    "sex",
    "age",
    pl.col("fare").alias("fare_gbp")
])

survived,passenger_class,sex,age,fare_gbp
i64,i64,str,f64,f64
0,3,"""male""",22.0,7.25
1,1,"""female""",38.0,71.2833
1,3,"""female""",26.0,7.925
1,1,"""female""",35.0,53.1
0,3,"""male""",35.0,8.05
…,…,…,…,…
0,2,"""male""",27.0,13.0
1,1,"""female""",19.0,30.0
0,3,"""female""",null,23.45



## 4) Filter rows and Sort rows


In [4]:
# Example: females in 1st/2nd class, sorted by fare descending
titanic.filter(
    (pl.col("sex") == "female") & (pl.col("pclass") <= 2)
).sort("fare", descending=True)

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
259,1,1,"""Ward, Miss. Anna""","""female""",35.0,0,0,"""PC 17755""",512.3292,null,"""C"""
89,1,1,"""Fortune, Miss. Mabel Helen""","""female""",23.0,3,2,"""19950""",263.0,"""C23 C25 C27""","""S"""
342,1,1,"""Fortune, Miss. Alice Elizabeth""","""female""",24.0,3,2,"""19950""",263.0,"""C23 C25 C27""","""S"""
312,1,1,"""Ryerson, Miss. Emily Borie""","""female""",18.0,2,2,"""PC 17608""",262.375,"""B57 B59 B63 B66""","""C"""
743,1,1,"""Ryerson, Miss. Susan Parker ""S…","""female""",21.0,2,2,"""PC 17608""",262.375,"""B57 B59 B63 B66""","""C"""
…,…,…,…,…,…,…,…,…,…,…,…
459,1,2,"""Toomey, Miss. Ellen""","""female""",50.0,0,0,"""F.C.C. 13531""",10.5,null,"""S"""
517,1,2,"""Lemore, Mrs. (Amelia Milley)""","""female""",34.0,0,0,"""C.A. 34260""",10.5,"""F33""","""S"""
527,1,2,"""Ridsdale, Miss. Lucy""","""female""",50.0,0,0,"""W./C. 14258""",10.5,null,"""S"""



## 5) `with_columns()` to create new columns using Expressions

`dataframe_name.with_columns()` will add columns to the data frame.

Provide a list of newly defined columns using expressions.

Name the columns with `.alias()`

One of the example expressions below is a `when-then-otherwise`

Learn more at: https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.when.html

we use `.lit()` to return a literal value, otherwise, Polars will look for a column called "adult" or a column called "child"

In [5]:
titanic_augmented = titanic.with_columns([
    (pl.col("sibsp") + pl.col("parch")).alias("family_size"),
    pl.when(pl.col("age") >= 18)
      .then(pl.lit("adult"))
      .otherwise(pl.lit("child"))
      .alias("age_group")
])
titanic_augmented

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,family_size,age_group
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,i64,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""",1,"""adult"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""",1,"""adult"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S""",0,"""adult"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""",1,"""adult"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S""",0,"""adult"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
887,0,2,"""Montvila, Rev. Juozas""","""male""",27.0,0,0,"""211536""",13.0,null,"""S""",0,"""adult"""
888,1,1,"""Graham, Miss. Margaret Edith""","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S""",0,"""adult"""
889,0,3,"""Johnston, Miss. Catherine Hele…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S""",3,"""child"""


### Window Expressions

`.over()` will compute an expression over given groups.

See the example below where mean is calculated for all passengers, and then calculated for each passenger class

In [6]:
titanic.with_columns(
    pl.col("fare").mean().alias("mean_fare"),
    pl.col("fare").mean().over("pclass").alias("mean_fare_by_pclass")
)

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,mean_fare,mean_fare_by_pclass
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,f64,f64
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""",32.204208,13.67555
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""",32.204208,84.154687
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S""",32.204208,13.67555
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""",32.204208,84.154687
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S""",32.204208,13.67555
…,…,…,…,…,…,…,…,…,…,…,…,…,…
887,0,2,"""Montvila, Rev. Juozas""","""male""",27.0,0,0,"""211536""",13.0,null,"""S""",32.204208,20.662183
888,1,1,"""Graham, Miss. Margaret Edith""","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S""",32.204208,84.154687
889,0,3,"""Johnston, Miss. Catherine Hele…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S""",32.204208,13.67555



## 6) Handling Missing Data

Use `is_null`, `is_not_null` to identify missing values.

Use `fill_null` to fill missing values. This can be paired with `.over()` to fill with averages calculated for groups.


In [7]:
# Identify missing ages
missing_age = titanic.select([
    "passengerid",
    "age",
    pl.col("age").is_null().alias("age_is_missing")
])
missing_age

passengerid,age,age_is_missing
i64,f64,bool
1,22.0,false
2,38.0,false
3,26.0,false
4,35.0,false
5,35.0,false
…,…,…
887,27.0,false
888,19.0,false
889,null,true


In [8]:
# Example: fill missing age with class/sex mean
filled_age = titanic.with_columns(
    pl.col("age").fill_null(
        pl.col("age").mean().over(["pclass","sex"])
    ).alias("age_filled")
)
filled_age

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,age_filled
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,f64
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""",22.0
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""",38.0
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S""",26.0
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""",35.0
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S""",35.0
…,…,…,…,…,…,…,…,…,…,…,…,…
887,0,2,"""Montvila, Rev. Juozas""","""male""",27.0,0,0,"""211536""",13.0,null,"""S""",27.0
888,1,1,"""Graham, Miss. Margaret Edith""","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S""",19.0
889,0,3,"""Johnston, Miss. Catherine Hele…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S""",21.75



## 7) Grouped Aggregations

Use `.group_by()` to create groups. Then us `.agg()` to specify expressions for aggregate (summary) values.


In [9]:
titanic.group_by(["pclass","sex"]).agg([
    pl.col("survived").mean().alias("survival_rate"),
    pl.col("fare").mean().alias("avg_fare"),
    pl.col("age").mean().alias("avg_age"),
    pl.len().alias("n")
]).sort(["pclass","sex"])

pclass,sex,survival_rate,avg_fare,avg_age,n
i64,str,f64,f64,f64,u32
1,"""female""",0.968085,106.125798,34.611765,94
1,"""male""",0.368852,67.226127,41.281386,122
2,"""female""",0.921053,21.970121,28.722973,76
2,"""male""",0.157407,19.741782,30.740707,108
3,"""female""",0.5,16.11881,21.75,144
3,"""male""",0.135447,12.661633,26.507589,347



## 8) Joins

You can join your data with another table.

Here we map `embarked` codes to city names.

https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

In [10]:
ports = pl.DataFrame({
    "embarked": ["C","Q","S"],
    "port_name": ["Cherbourg","Queenstown","Southampton"]
})

joined = titanic.join(ports, on="embarked", how="left")
joined

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,port_name
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S""","""Southampton"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C""","""Cherbourg"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S""","""Southampton"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S""","""Southampton"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S""","""Southampton"""
…,…,…,…,…,…,…,…,…,…,…,…,…
887,0,2,"""Montvila, Rev. Juozas""","""male""",27.0,0,0,"""211536""",13.0,null,"""S""","""Southampton"""
888,1,1,"""Graham, Miss. Margaret Edith""","""female""",19.0,0,0,"""112053""",30.0,"""B42""","""S""","""Southampton"""
889,0,3,"""Johnston, Miss. Catherine Hele…","""female""",null,1,2,"""W./C. 6607""",23.45,null,"""S""","""Southampton"""


## 9) Reshaping: Pivot and Unpivot

Compute a **survival rate** table by `sex and pclass`.

We then pivot it (equivalent to pivot_wide), then unpivot it back (pivot long).


In [11]:
# First, we make a summary table of survival rate by sex and pclass
survival = (
    titanic.group_by(["sex","pclass"])
           .agg(pl.col("survived").mean().alias("survival_rate"))
).sort(['pclass','sex'])
survival

sex,pclass,survival_rate
str,i64,f64
"""female""",1,0.968085
"""male""",1,0.368852
"""female""",2,0.921053
"""male""",2,0.157407
"""female""",3,0.5
"""male""",3,0.135447


In [12]:
# arguments:
# on is the column name that contains the values that will become column headings
survival_pivot = survival.pivot(
        on="pclass",
        index="sex",
        values="survival_rate"
        )
survival_pivot

sex,1,2,3
str,f64,f64,f64
"""female""",0.968085,0.921053,0.5
"""male""",0.368852,0.157407,0.135447


In [13]:
# unpivot is like pivot_longer
# on takes a list of column names that will be unpivoted
survival_long = survival_pivot.unpivot(on=['1','2','3'], index = "sex", variable_name="pclass", value_name="survival_rate")
survival_long

sex,pclass,survival_rate
str,str,f64
"""female""","""1""",0.968085
"""male""","""1""",0.368852
"""female""","""2""",0.921053
"""male""","""2""",0.157407
"""female""","""3""",0.5
"""male""","""3""",0.135447



## 11) Lazy Optimizations (Predicate/Projection Pushdown)


In [14]:
# Lazy scan of the CSV file
titanic_lazy = pl.scan_csv("titanic.csv")

lazy_query = (
  titanic_lazy
  .select(["survived","pclass","sex","fare"])
  .filter((pl.col("fare") > 30) & (pl.col("pclass") <= 2))
)


In [15]:
lazy_query # reveals Polars plan to run the query

In [16]:
lazy_query.collect()

survived,pclass,sex,fare
i64,i64,str,f64
1,1,"""female""",71.2833
1,1,"""female""",53.1
0,1,"""male""",51.8625
1,2,"""female""",30.0708
1,1,"""male""",35.5
…,…,…,…
1,1,"""female""",39.4
1,1,"""female""",164.8667
0,1,"""male""",50.4958


In [17]:
lazy_query2 = (
    titanic_lazy
    .filter(pl.col("age").is_not_null())
    .with_columns(
        (pl.col("age") >= 18).alias("adult"),
    )
    .group_by(["pclass", "sex"])
    .agg(
        survival_rate = pl.col("survived").mean(),
        avg_age       = pl.col("age").mean(),
        avg_fare      = pl.col("fare").mean(),
        prop_adult    = pl.col("adult").mean(),
        n             = pl.len()
    )
    .sort(["pclass", "sex"])
)

In [18]:
lazy_query2.collect()

pclass,sex,survival_rate,avg_age,avg_fare,prop_adult,n
i64,str,f64,f64,f64,f64,u32
1,"""female""",0.964706,34.611765,107.946275,0.905882,85
1,"""male""",0.39604,41.281386,71.142781,0.960396,101
2,"""female""",0.918919,28.722973,21.95107,0.837838,74
2,"""male""",0.151515,30.740707,21.113131,0.888889,99
3,"""female""",0.460784,21.75,15.875369,0.656863,102
3,"""male""",0.150198,26.507589,12.162695,0.83004,253
